In [1]:
%config Completer.use_jedi = False
import sys
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import time
import threading

GPU = False
import os

if GPU:
    txt_device = 'gpu:0'
else:
    txt_device = 'cpu:0'    
    os.environ["CUDA_VISIBLE_DEVICES"]="-1"


In [122]:
class NN(tf.keras.layers.Layer):
    
    def __init__(self, layers, **kwargs):
        """
        
           layers: input, dense layers and outputs dimensions  
        """
        super().__init__(**kwargs)
        self.layers = layers
        self.num_layers = len(self.layers)        
        
    def build(self, input_shape):         
        """Create the state of the layers (weights)"""
        weights = []
        biases = []        
        for l in range(0,self.num_layers-1):
            W = self.xavier_init(size=[self.layers[l], self.layers[l+1]])
            b = tf.Variable(tf.zeros([1,self.layers[l+1]], dtype=tf.float32), dtype=tf.float32)
            weights.append(W)
            biases.append(b)
        
        self.Ws = weights
        self.bs = biases
        
    def xavier_init(self, size):
        in_dim = size[0]
        out_dim = size[1]        
        xavier_stddev = np.sqrt(2/(in_dim + out_dim))
        return tf.Variable(tf.compat.v1.truncated_normal([in_dim, out_dim], stddev=xavier_stddev), 
                           dtype=tf.float32)
    
    def normalise_input(self, inputs):
        """Map the inputs to the range [-1, 1]"""
        return 2.0*(inputs - self.lb)/(self.ub - self.lb) - 1.0
    
    def __net__(self, inputs):
        H = inputs   
        for W,b in zip(self.Ws[:-1], self.bs[:-1]):
            H = tf.tanh(tf.add(tf.matmul(H, W), b))
            
            W = self.Ws[-1]
            b = self.bs[-1]
            outputs = tf.add(tf.matmul(H, W), b)
        return outputs
    
    def call(self, inputs, grads=True):
        """Defines the computation from inputs to outputs"""
        
                                                     
                        
        if grads:
            with tf.GradientTape(persistent=False) as second_order_t:
                second_order_t.watch(inputs)
                with tf.GradientTape(persistent=False) as first_order_t:
                    first_order_t.watch(inputs)                
                    outputs = self.__net__(inputs)
                
                    first_orders = first_order_t.gradient(outputs, inputs)        
                    second_orders = second_order_t.gradient(first_orders, inputs)
            return outputs, first_orders, second_orders
        else:
            outputs = self.__net__(inputs)
            return outputs
    
    def grads(self):
        first_orders = self.first_order_t.gradient(self.Y, self.X)        
        secnd_orders = self.second_order_t.gradient(first_orders, self.X)
        return first_orders, secnd_orders

In [123]:
def lower_upper_bounds(inputs_of_inputs):
    """Find the lower and upper bounds of inputs
    
       inputs_of_inputs: a list of tensors that their axis one have the same number 
                         of columns
    """
           
    inputs_dim = np.asarray(inputs_of_inputs[0]).shape[1]
    lb = np.array([np.inf] * inputs_dim)
    ub = np.array([-np.inf] * inputs_dim)
    for i, inputs in enumerate(inputs_of_inputs):        
        assert inputs_dim == np.asarray(inputs).shape[1]
        lb = np.amin(np.c_[inputs.min(0), lb], 1)
        ub = np.amax(np.c_[inputs.max(0), ub], 1)
        
    return lb, ub
    
def normalise_inputs(inputs_of_inputs):
    """Scales the values along axis 1 to [-1, 1]
    
       inputs_of_inputs: a list of tensors that their axis one have the same number 
                         of columns
    """
    if type(inputs_of_inputs) is not list:
        inputs_of_inputs = [inputs_of_inputs]        
            
    lb, ub = lower_upper_bounds(inputs_of_inputs)
    return [2.0*(inputs-lb)/(ub-lb) - 1.0 for inputs in inputs_of_inputs]    

In [124]:
layers = [3, 10, 10, 10, 2]
sample_xyt  = np.array([ [0, 0, 0],
                         [0, 1, 0],
                         [1, 0, 0],
                         [1, 1, 0],
                         [0, 0, 1],
                         [0, 1, 1],
                         [1, 0, 1],
                         [1, 1, 1],
                         [0, 0, 2],
                         [0, 1, 2],
                         [1, 0, 2],
                         [1, 1, 2]
                       ])

#lb, ub = lower_upper_bounds(sample_xyt)
#print(normalise_inputs([sample_xyt, sample_xyt.copy()*2]))

model = NN(layers)

In [127]:
i = tf.constant(sample_xyt*1.0)
o = model(i, True)

In [128]:
o

(<tf.Tensor: shape=(12, 2), dtype=float32, numpy=
 array([[ 0.        ,  0.        ],
        [-0.08767529, -0.21518287],
        [-0.02415903, -0.08333324],
        [-0.16224366, -0.27726024],
        [ 0.4444737 , -0.36355215],
        [ 0.28590775, -0.52748233],
        [ 0.30637497, -0.3949511 ],
        [ 0.16782539, -0.58145547],
        [ 0.65741426, -0.5023011 ],
        [ 0.5166262 , -0.6460754 ],
        [ 0.47272864, -0.5192702 ],
        [ 0.36170444, -0.6895684 ]], dtype=float32)>,
 <tf.Tensor: shape=(12, 3), dtype=float32, numpy=
 array([[-0.20433848, -0.33760384,  0.08058533],
        [-0.20068434, -0.24486914,  0.01957287],
        [ 0.01540439, -0.33235598,  0.02412978],
        [-0.01175655, -0.28502095, -0.00749357],
        [-0.20690554, -0.30236012,  0.0807012 ],
        [-0.17983906, -0.29743984,  0.09972762],
        [-0.07386359, -0.29976797,  0.02648415],
        [-0.07915174, -0.31036854,  0.06210135],
        [-0.18094352, -0.252105  ,  0.06414592],
        [